<a href="https://colab.research.google.com/github/esakik/curtin/blob/main/reinforcement_learning/multi_armed_bandits_%CE%B5_greedy_optimism_non_stationary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#######################################################################
# Copyright (C)                                                       #
# 2016-2018 Shangtong Zhang(zhangshangtong.cpp@gmail.com)             #
# 2016 Tian Jun(tianjun.cpp@gmail.com)                                #
# 2016 Artem Oboturov(oboturov@gmail.com)                             #
# 2016 Kenta Shimada(hyperkentakun@gmail.com)                         #
# Permission given to modify the code as long as you keep this        #
# declaration at the top                                              #
#######################################################################

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [2]:
class MABEnv:

    def __init__(self, n_arms=10, mean=0.0):
        """Initialize Multi-armed Bandits (MAB) Environment class.

        Args:
            n_arms: Number of arms to pull.
            mean: Mean value for normal distributions.
        """
        self.n_arms = n_arms
        self.mean = mean

    def reset(self):
        """Reset all the information."""
        # True reward drawn from the normal distribution with mean for each action
        self.q_true = np.random.randn(self.n_arms) + self.mean

        # Action index with best true reward
        self.optimal_action = np.argmax(self.q_true)

    def step(self, action):
        """Take action and return reward.

        Args:
            action: Index of next action

        Returns:
            reward
        """
        # Stochastic reward drawn from the normal distribution
        return np.random.randn() + self.q_true[action]

    def reward_distribution_example():
        plt.violinplot(dataset=np.random.randn(200, 10) + np.random.randn(10))
        plt.xlabel("Action")
        plt.ylabel("Reward distribution")
        plt.savefig('./reward_distribution_example.png')
        plt.close()

In [3]:
class EpsilonGreedyAgent:

    def __init__(self, env, alpha=None, initial=0.5, epsilon=0.1):
        """Initialize ε-greedy Agent class.

        Args:
            env: Multi-armed Bandits (MAB) Environment class.
            alpha: Step size parameter to value the latest estimation
            initial: Optimistic initial values
            epsilon: Probability for exploration.
        """
        self.env = env
        self.alpha = alpha
        self.initial = initial
        self.epsilon = epsilon

    def reset(self):
        """Reset all the information."""
        # Estimated reward for each action
        self.q_estimation = np.zeros(self.env.n_arms) + self.initial

        # Chosen times for each action
        self.action_count = np.zeros(self.env.n_arms)

    def policy(self):
        """Get index of next action.

        Returns:
            Index of next action
        """
        if np.random.rand() < self.epsilon:
            return np.random.choice(np.arange(self.env.n_arms))
        return np.argmax(self.q_estimation)

    def update(self, action, reward):
        """Update estimation.

        Args:
            action: Index of next action
            reward: Reward from environment
        """
        if self.alpha is None:
            # Q(A)←Q(A)+1/N(A)[R−Q(A)]
            self.action_count[action] += 1
            self.q_estimation[action] += 1.0 / self.action_count[action] * (reward - self.q_estimation[action])
        else:
            # Qn+1=Qn+α[Rn−Qn]
            self.q_estimation[action] += self.alpha * (reward - self.q_estimation[action])

In [4]:
class Simulator:

    def __init__(self, agents):
        self.agents = agents

    def run(self, runs=2000, steps=3000):
        optimal_action_counts, rewards = self._run(self.agents, runs, steps)

        plt.figure(figsize=(10, 20))

        plt.subplot(2, 1, 1)
        for eps, rewards in zip(epsilons, rewards):
            plt.plot(rewards, label='epsilon = %.02f' % (eps))
        plt.xlabel('steps')
        plt.ylabel('average reward')
        plt.legend()

        plt.subplot(2, 1, 2)
        for eps, counts in zip(epsilons, optimal_action_counts):
            plt.plot(counts, label='epsilon = %.02f' % (eps))
        plt.xlabel('steps')
        plt.ylabel('% optimal action')
        plt.legend()

        plt.savefig('./simulation_result.png')
        plt.close()

    def _run(self, agents, runs, steps):
        optimal_action_counts = np.zeros((len(agents), runs, steps))
        rewards = np.zeros(optimal_action_counts.shape)

        for i, agent in enumerate(agents):
            for r in tqdm(range(runs)):
                agent.env.reset()
                agent.reset()

                for s in range(steps):
                    action = agent.policy()
                    reward = agent.env.step(action=action)
                    agent.update(action=action, reward=reward)

                    rewards[i, r, s] = reward
                    if action == agent.env.optimal_action:
                        optimal_action_counts[i, r, s] = 1

        optimal_action_counts = optimal_action_counts.mean(axis=1)
        rewards = rewards.mean(axis=1)

        return optimal_action_counts, rewards

In [5]:
env = MABEnv(n_arms=10, mean=0.0)
epsilons = [0, 0.1, 0.01, 0.5, 1.0]
agents = [EpsilonGreedyAgent(env=env, epsilon=epsilon) for epsilon in epsilons]
simulator = Simulator(agents=agents)
simulator.run(runs=2000, steps=3000)

100%|██████████| 2000/2000 [01:47<00:00, 18.58it/s]
